<a href="https://colab.research.google.com/github/yoseforaz0990/ML-templates/blob/main/deep_learning/Generative_Adversarial_Networks%20(GAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Step Name                                          | Description                                                                                             |
|----------------------------------------------------|---------------------------------------------------------------------------------------------------------|
| Importing the Dataset                             | Loading the MNIST dataset, which consists of images of handwritten digits (0 to 9).                  |
| Filtering out the Data for Faster Training        | Filtering only the images corresponding to digit '0' for faster training on a smaller dataset.       |
| Importing necessary libraries                     | Importing the required libraries for building the GAN.                                                |
| Setting random seeds for reproducibility          | Setting random seeds to ensure reproducibility of the results.                                        |
| Defining the size of input noise vector           | Specifying the size of the input noise vector for the generator.                                      |
| Creating the Generator model                      | Defining the generator model architecture using fully connected layers.                               |
| Creating the Discriminator model                  | Defining the discriminator model architecture using fully connected layers.                           |
| Compiling the Discriminator model                 | Compiling the discriminator model with binary cross-entropy loss and Adam optimizer.                  |
| Combining the Generator and Discriminator         | Combining the generator and discriminator models to create the GAN model.                             |
| Making the Discriminator not trainable            | Setting the discriminator to be non-trainable in the GAN model.                                       |
| Compiling the GAN model                           | Compiling the GAN model with binary cross-entropy loss and Adam optimizer.                             |
| Setting up Training Batches                        | Preparing the data in batches for efficient training.                                                 |
| Specifying the number of epochs for training       | Specifying the number of epochs to train the GAN.                                                      |
| Training Loop                                     | The main training loop for the GAN.                                                                    |
| Generating images using the trained Generator      | Generating fake images using the trained Generator.                                                    |


In [ ]:
# Step 1: Importing the Dataset
# Description: Loading the MNIST dataset, which consists of images of handwritten digits (0 to 9).

# Loading MNIST dataset
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Step 2: Filtering out the Data for Faster Training on Smaller Dataset
# Description: Filtering out only the images corresponding to digit '0' for faster training on a smaller dataset.

# Filtering only the images corresponding to digit '0'
only_zeros = X_train[y_train == 0]
only_zeros.shape

# Step 3: Importing necessary libraries
# Description: Importing the required libraries for building the GAN.

# Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten
from tensorflow.keras.models import Sequential

# Step 4: Setting random seeds for reproducibility
# Description: Setting random seeds to ensure reproducibility of the results.

# Setting random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Step 5: Defining the size of input noise vector
# Description: Specifying the size of the input noise vector for the generator.

# Defining the size of input noise vector
codings_size = 100

# Step 6: Creating the Generator model
# Description: Defining the generator model architecture using fully connected layers.

# Creating the Generator model
generator = Sequential()
generator.add(Dense(100, activation="relu", input_shape=[codings_size]))
generator.add(Dense(150, activation='relu'))
generator.add(Dense(784, activation="sigmoid"))  # 28*28 = 784
generator.add(Reshape([28, 28]))

# Step 7: Creating the Discriminator model
# Description: Defining the discriminator model architecture using fully connected layers.

# Creating the Discriminator model
discriminator = Sequential()
discriminator.add(Flatten(input_shape=[28, 28]))
discriminator.add(Dense(150, activation='relu'))
discriminator.add(Dense(100, activation='relu'))
discriminator.add(Dense(1, activation="sigmoid"))

# Step 8: Compiling the Discriminator model
# Description: Compiling the discriminator model with binary cross-entropy loss and Adam optimizer.

# Compiling the Discriminator model
discriminator.compile(loss="binary_crossentropy", optimizer="adam")

# Step 9: Combining the Generator and Discriminator to create the GAN model
# Description: Combining the generator and discriminator models to create the GAN model.

# Combining the Generator and Discriminator to create the GAN model
GAN = Sequential([generator, discriminator])

# Step 10: Making the Discriminator not trainable in the combined GAN model
# Description: Setting the discriminator to be non-trainable in the GAN model.

# Making the Discriminator not trainable in the combined GAN model
discriminator.trainable = False

# Step 11: Compiling the GAN model
# Description: Compiling the GAN model with binary cross-entropy loss and Adam optimizer.

# Compiling the GAN model
GAN.compile(loss="binary_crossentropy", optimizer="adam")

# Step 12: Setting up Training Batches
# Description: Preparing the data in batches for efficient training.

# Setting the batch size for training
batch_size = 32

# Using only_zeros dataset for training
my_data = only_zeros

# Creating TensorFlow Dataset from the filtered dataset
dataset = tf.data.Dataset.from_tensor_slices(my_data).shuffle(buffer_size=1000)

# Dividing the dataset into batches and prefetching for efficiency
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

# Step 13: Specifying the number of epochs for training
# Description: Specifying the number of epochs to train the GAN.

# Specifying the number of epochs for training
epochs = 1

# Step 14: Training Loop
# Description: The main training loop for the GAN.

# Grabbing the separate components (Generator and Discriminator) from the GAN model
generator, discriminator = GAN.layers

# Training loop over multiple epochs
for epoch in range(epochs):
    print(f"Currently on Epoch {epoch + 1}")
    i = 0
    # For every batch in the dataset
    for X_batch in dataset:
        i = i + 1
        if i % 100 == 0:
            print(f"\tCurrently on batch number {i} of {len(my_data) // batch_size}")

        # Training the Discriminator
        # Creating noise for Generator input
        noise = tf.random.normal(shape=[batch_size, codings_size])

        # Generating fake images using the Generator
        gen_images = generator(noise)

        # Concatenating real and fake images to train Discriminator
        X_fake_vs_real = tf.concat([gen_images, tf.dtypes.cast(X_batch, tf.float32)], axis=0)

        # Setting target labels for Discriminator training
        y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)

        # Making the Discriminator trainable for this step
        discriminator.trainable = True

        # Training the Discriminator on the batch
        discriminator.train_on_batch(X_fake_vs_real, y1)

        # Training the Generator
        # Creating new noise for Generator input
        noise = tf.random.normal(shape=[batch_size, codings_size])

        # Setting target labels for Generator training (to fool Discriminator)
        y2 = tf.constant([[1.]] * batch_size)

        # Making the Discriminator not trainable for this step
        discriminator.trainable = False

        # Training the GAN on the batch
        GAN.train_on_batch(noise, y2)

# Training complete
print("TRAINING COMPLETE")

# Step 15: Generating images using the trained Generator
# Description: Generating fake images using the trained Generator.

# Creating noise for Generator input
noise = tf.random.normal(shape=[10, codings_size])

# Generating fake images using the Generator
image = generator(noise)

# Displaying one of the generated images
plt.imshow(image[5])
